#### Importando bibliotecas

In [1]:
from pymongo import MongoClient
import requests
import pandas as pd

#### Conectando ao MongoDB Atlas

In [ ]:
# Conectar ao MongoDB Atlas
client = MongoClient("")
# Seleciona o banco de dados
db = client[""]
# Seleciona a coleção
collection = db["estoque_e_transacoes_cartoes"]

In [3]:
# Recuperar dados da coleção
data = list(collection.find())
data

[]

In [4]:
# Converter para DataFrame
df = pd.DataFrame(data)
print(df)

Empty DataFrame
Columns: []
Index: []


#### Consumindo API e enviando dados para o banco

In [5]:
# Endpoint da nova API para os dados de meios de pagamento
url = ("https://olinda.bcb.gov.br/olinda/servico/MPV_DadosAbertos/versao/v1/odata/Quantidadeetransacoesdecartoes(trimestre=@trimestre)?@trimestre='20201'&$top=10000&$format=json&$select=trimestre,nomeBandeira,nomeFuncao,produto,qtdCartoesEmitidos,qtdCartoesAtivos,qtdTransacoesNacionais,valorTransacoesNacionais,qtdTransacoesInternacionais,valorTransacoesInternacionais")

# Realizar a requisição
response = requests.get(url)

# Verificar se a requisição foi bem-sucedida
if response.status_code == 200:
    data = response.json()

    # Inicializar a lista para armazenar os dados
    pagamento_data = []

    # Iterar pelos resultados
    for item in data["value"]:

        # Extrair o trimestre e ano
        trimestre_str = item["trimestre"]
        ano = trimestre_str[:4]  # Os primeiros 4 caracteres representam o ano
        trimestre = int(trimestre_str[4:])  # Os últimos 1 ou 2 caracteres representam o trimestre

        # Determinar o nome do trimestre
        if trimestre == 1:
            nome_trimestre = "1º Trimestre"
        elif trimestre == 2:
            nome_trimestre = "2º Trimestre"
        elif trimestre == 3:
            nome_trimestre = "3º Trimestre"
        else:
            nome_trimestre = "4º Trimestre"

        # Detalhes da transação e quantidade de cartões emitidos e ativos
        # Vamos organizar os dados por bandeira, função e produto
        pagamento_data.append({
            "ano": ano,
            "trimestre": nome_trimestre,
            "bandeira": item["nomeBandeira"],
            "funcao": item["nomeFuncao"],
            "produto": item["produto"],
            "quantidade_cartoes_emitidos": item["qtdCartoesEmitidos"],
            "quantidade_cartoes_ativos": item["qtdCartoesAtivos"],
            "qtd_transacoes_nacionais": item["qtdTransacoesNacionais"],
            "valor_transacoes_nacionais": item["valorTransacoesNacionais"],
            "qtd_transacoes_internacionais": item["qtdTransacoesInternacionais"],
            "valor_transacoes_internacionais": item["valorTransacoesInternacionais"]
        })

else:
    print("Erro:", response.status_code, response.text)


In [6]:
# Converter para DataFrame
df = pd.DataFrame(pagamento_data)
df.head()

,ano,trimestre,bandeira,funcao,produto,quantidade_cartoes_emitidos,quantidade_cartoes_ativos,qtd_transacoes_nacionais,valor_transacoes_nacionais,qtd_transacoes_internacionais,valor_transacoes_internacionais
0,2021,3º Trimestre,Bandeira própria,Crédito,Básico Nacional,1938972,535612,2741493,5.213396e+08,0,0.000000e+00
1,2023,2º Trimestre,VISA,Crédito,Empresarial,3050384,1716709,43984902,1.284661e+10,470796,3.970433e+08
2,2024,3º Trimestre,VISA,Pré-Pago,Empresarial,1548960,453317,2979851,1.665969e+08,6144,1.607533e+06
3,2021,1º Trimestre,MasterCard,Débito,Básico Internacional,83088117,21554284,567370086,3.906461e+10,27511,9.502222e+06
4,2021,1º Trimestre,MasterCard,Crédito,Premium,2197074,1785104,107175156,2.110709e+10,1234605,6.312240e+08


In [7]:
#tranformar dados em dicionário para insertir no banco de dados
data_dict = df.to_dict("records")
print(data_dict)

[{'ano': '2021', 'trimestre': '3º Trimestre', 'bandeira': 'Bandeira própria', 'funcao': 'Crédito', 'produto': 'Básico Nacional', 'quantidade_cartoes_emitidos': 1938972, 'quantidade_cartoes_ativos': 535612, 'qtd_transacoes_nacionais': 2741493, 'valor_transacoes_nacionais': 521339586.1, 'qtd_transacoes_internacionais': 0, 'valor_transacoes_internacionais': 0.0}, {'ano': '2023', 'trimestre': '2º Trimestre', 'bandeira': 'VISA', 'funcao': 'Crédito', 'produto': 'Empresarial', 'quantidade_cartoes_emitidos': 3050384, 'quantidade_cartoes_ativos': 1716709, 'qtd_transacoes_nacionais': 43984902, 'valor_transacoes_nacionais': 12846611557.78, 'qtd_transacoes_internacionais': 470796, 'valor_transacoes_internacionais': 397043258.04}, {'ano': '2024', 'trimestre': '3º Trimestre', 'bandeira': 'VISA', 'funcao': 'Pré-Pago', 'produto': 'Empresarial', 'quantidade_cartoes_emitidos': 1548960, 'quantidade_cartoes_ativos': 453317, 'qtd_transacoes_nacionais': 2979851, 'valor_transacoes_nacionais': 166596853.06, '

In [8]:
# Inserindo os dados no MongoDB
collection.insert_many(data_dict)

InsertManyResult([ObjectId('6757175502d6b4e52d125dcd'), ObjectId('6757175502d6b4e52d125dce'), ObjectId('6757175502d6b4e52d125dcf'), ObjectId('6757175502d6b4e52d125dd0'), ObjectId('6757175502d6b4e52d125dd1'), ObjectId('6757175502d6b4e52d125dd2'), ObjectId('6757175502d6b4e52d125dd3'), ObjectId('6757175502d6b4e52d125dd4'), ObjectId('6757175502d6b4e52d125dd5'), ObjectId('6757175502d6b4e52d125dd6'), ObjectId('6757175502d6b4e52d125dd7'), ObjectId('6757175502d6b4e52d125dd8'), ObjectId('6757175502d6b4e52d125dd9'), ObjectId('6757175502d6b4e52d125dda'), ObjectId('6757175502d6b4e52d125ddb'), ObjectId('6757175502d6b4e52d125ddc'), ObjectId('6757175502d6b4e52d125ddd'), ObjectId('6757175502d6b4e52d125dde'), ObjectId('6757175502d6b4e52d125ddf'), ObjectId('6757175502d6b4e52d125de0'), ObjectId('6757175502d6b4e52d125de1'), ObjectId('6757175502d6b4e52d125de2'), ObjectId('6757175502d6b4e52d125de3'), ObjectId('6757175502d6b4e52d125de4'), ObjectId('6757175502d6b4e52d125de5'), ObjectId('6757175502d6b4e52d125d